In [ ]:
!pip install firebase-admin requests -q

In [ ]:
from google.colab import files
print('Please upload your firebase_key.json file:')
uploaded = files.upload()
print('File uploaded successfully!')

In [ ]:
import requests
import json
from datetime import datetime
import firebase_admin
from firebase_admin import credentials, db

# Firebase initialization
if not firebase_admin._apps:
    cred = credentials.Certificate('firebase_key.json')
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://cloud-81451-default-rtdb.europe-west1.firebasedatabase.app/'
    })
    print('Firebase initialized')

# Configuration
BASE_URL = "https://server-cloud-v645.onrender.com/"
FEED, BATCH_LIMIT = "json", 200
EARLIEST_DATE = "2025-10-01T00:00:00Z"

def get_latest_timestamp():
    """Get the most recent timestamp from Firebase"""
    try:
        latest = db.reference('/sensor_data').order_by_child('created_at').limit_to_last(1).get()
        return list(latest.values())[0]['created_at'] if latest else None
    except Exception as e:
        print(f"Error: {e}")
        return None

def fetch_batch(before=None):
    """Fetch batch from API"""
    params = {"feed": FEED, "limit": BATCH_LIMIT}
    if before:
        params["before_created_at"] = before
    return requests.get(f"{BASE_URL}/history", params=params).json()

def save_to_firebase(data_list):
    """Save data to Firebase"""
    ref = db.reference('/sensor_data')
    for i, sample in enumerate(data_list, 1):
        values = json.loads(sample['value'])
        ref.child(sample['created_at'].replace(':', '-').replace('.', '-')).set({
            'created_at': sample['created_at'],
            'temperature': values['temperature'],
            'humidity': values['humidity'],
            'soil': values['soil']
        })
        if i % 100 == 0:
            print(f"  Saved {i}/{len(data_list)}...")
    return len(data_list)

def download_all():
    """Download all historical data"""
    print(f"Downloading from {EARLIEST_DATE}...")
    response = fetch_batch()
    if "data" not in response:
        print("Error:", response)
        return []
    
    all_data = response["data"]
    print(f"Fetched {len(all_data)} initial samples")
    
    prev_oldest, stuck = None, 0
    for batch_num in range(1, 1001):
        if not all_data or all_data[-1]["created_at"] <= EARLIEST_DATE:
            break
        
        before = all_data[-1]["created_at"]
        stuck = stuck + 1 if before == prev_oldest else 0
        if stuck >= 2:
            print("No more data")
            break
        prev_oldest = before
        
        response = fetch_batch(before)
        if "data" not in response or not response["data"]:
            break
        
        filtered = [s for s in response["data"] if EARLIEST_DATE <= s["created_at"] < before]
        if filtered:
            all_data.extend(filtered)
            print(f"Batch {batch_num}: {len(filtered)} samples. Total: {len(all_data)}")
        else:
            break
    
    return [s for s in all_data if s["created_at"] >= EARLIEST_DATE]

def download_new(latest):
    """Download data newer than latest timestamp"""
    print(f"Downloading newer than {latest}...")
    response = fetch_batch()
    if "data" not in response:
        print("Error:", response)
        return []
    
    new_data = [s for s in response["data"] if s["created_at"] > latest]
    print(f"Found {len(new_data)} new samples")
    return new_data

def main():
    print("=" * 60)
    print(f"SENSOR FIREBASE SYNC - {datetime.now().isoformat()}")
    print("=" * 60)
    
    latest = get_latest_timestamp()
    data = download_new(latest) if latest else download_all()
    
    if data:
        if not latest:
            print(f"\nTotal: {len(data)} | Range: {data[-1]['created_at']} to {data[0]['created_at']}")
        print(f"\nSaving {len(data)} records...")
        saved = save_to_firebase(data)
        print(f"\n{saved} records saved!")
    else:
        print("\nNo new data. Database up to date!")
    
    print("\n" + "=" * 60)
    print("SYNC COMPLETE")
    print("=" * 60)

print('Functions loaded. Ready to run!')

In [ ]:
main()